In [123]:
%load_ext autoreload
%autoreload 2  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
frames_water[0].info

{'TotEnergy': -30025.003855199237, 'cutoff': -1.0, 'nneightol': 1.2}

In [40]:
import random
import sys
import os
import ase.io

sys.path.append(os.path.join(os.getcwd(), "..", "model"))
sys.path.append(os.path.join(os.getcwd(), "..", "train"))
sys.path.append(os.path.join(os.getcwd(), "..", "utils"))

from pytorch_lightning import Trainer
from rascaline_trainer import BPNNRascalineModule
from load import load_PBE0_TS
from dataset.dataset import create_rascaline_dataloader
import rascaline
import torch
from transformer.composition import CompositionTransformer
import equistore
from pytorch_lightning.loggers import CSVLogger
from nn.model import BPNNModel


#default type is float64
torch.set_default_dtype(torch.float64)

# --- load the data ---
frames_water = ase.io.read("../data/water_converted.xyz", index=":")
random.shuffle(frames_water)

frames_water = frames_water[:10]
frames_train = frames_water[:5]
frames_val = frames_water[5:]

# --- define the hypers ---
hypers_ps = {
    "cutoff": 5.0,
    "max_radial": 5,
    "max_angular": 5,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width":0.5},
    },
    "radial_scaling":{"Willatt2018": {"exponent": 3.0, "rate": 1.5, "scale": 2.0}}
}

hypers_rs = {
    "cutoff": 5.0,
    "max_radial": 12,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width":0.5},
    },
    "radial_scaling":{"Willatt2018": {"exponent": 3.0, "rate": 1.5, "scale": 2.0}}
}

# --- define calculator ---
calc_ps = rascaline.torch.SoapPowerSpectrum(**hypers_ps)
calc_rs = rascaline.torch.SoapRadialSpectrum(**hypers_rs)


dataloader_setup = create_rascaline_dataloader(frames_train,
                                         calculators=calc_ps,#setup,
                                         do_gradients=True,
                                         precompute = True,
                                         lazy_fill_up = False,
                                         batch_size=len(frames_train), 
                                         shuffle=False,
                                         energy_key="TotEnergy",
                                         forces_key="force",
                                         )

transformer = CompositionTransformer()
feat, prop, syst = next(iter(dataloader_setup))
#prop = equistore.to(prop,"torch",dtype=torch.float64)
transformer.fit(syst, prop)

del feat, prop, syst
del dataloader_setup




# --- create the dataloaders ---
dataloader_train = create_rascaline_dataloader(frames_train,
                                         calculators=[calc_rs,calc_ps],
                                         do_gradients=True,
                                         precompute = True,
                                         lazy_fill_up = False,
                                         batch_size=len(frames_train), 
                                         shuffle=False,
                                         energy_key="TotEnergy",
                                         forces_key="force",
                                         )
                                         #collate_fn=_equistor_collate_w_custom_idx)

dataloader_val = create_rascaline_dataloader(frames_val,
                                         calculators=[calc_rs,calc_ps],
                                         do_gradients=True,
                                         precompute =  True,
                                         lazy_fill_up = False,
                                         batch_size=len(frames_val),
                                         energy_key="TotEnergy",
                                         forces_key="force",
                                         shuffle=False)
                                         #ollate_fn=_equistor_collate_w_custom_idx)

# --- create the trainer ---
# for now a batch of features is necessary 
# for correct weight init

batch = next(iter(dataloader_train))
feat, prop, syst = batch

TypeError: SoapPowerSpectrum.__init__() missing 1 required positional argument: 'max_angular'

In [39]:
feat.block(0).values.shape

torch.Size([640, 450])

In [15]:
hypers_sr = {
    "cutoff": 5.0,
    "max_radial": 5,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width":0.5},
    },
    "radial_scaling":{"Willatt2018": {"exponent": 3.0, "rate": 1.5, "scale": 2.0}}
}

In [16]:
calc_2 = rascaline.SoapRadialSpectrum(**hypers_sr)

In [33]:
hypers_sr = {
    "cutoff": 5.0,
    "max_radial": 5,
    "max_angular": 5,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width":0.5},
    },
    "radial_scaling":{"Willatt2018": {"exponent": 3.0, "rate": 1.5, "scale": 2.0}}
}
calc_2 = rascaline.SoapPowerSpectrum(**hypers_sr)

In [34]:
feat = calc_2.compute(frames_train[0])

In [36]:
feat.keys.names

['species_center', 'species_neighbor_1', 'species_neighbor_2']

In [35]:
'species_neighbor' in feat.keys.names

False

In [32]:
feat.keys.names == ['species_center', 'species_neighbor']

True

In [25]:
feat, prop, syst = batch

In [26]:
'species_neighbor_1' in feat.keys.names

False

In [27]:
feat.keys.names

['species_center']

In [12]:
feat.block(0).values

tensor([[ 5.7172e-01, -1.6151e-01,  1.7040e-01,  ...,  1.1825e-04,
          3.7443e-04,  2.4462e-04],
        [ 6.3414e-01,  1.0310e-03,  3.4403e-01,  ...,  3.4460e-04,
          7.2606e-04,  2.5071e-04],
        [ 6.0876e-01, -1.1166e-01,  2.3695e-01,  ...,  2.1511e-04,
          2.2095e-04,  9.8346e-05],
        ...,
        [ 5.4764e-01, -9.4518e-02,  2.0309e-01,  ...,  6.4470e-04,
          4.2267e-04,  2.8083e-04],
        [ 6.2276e-01, -1.0161e-01,  2.2053e-01,  ...,  1.7590e-04,
          2.9594e-04,  1.1292e-04],
        [ 6.3974e-01, -8.4837e-02,  2.7116e-01,  ...,  6.2436e-04,
          5.1160e-04,  2.2617e-04]], grad_fn=<IndexPutBackward0>)

In [256]:
dataloader_train = create_rascaline_dataloader(frames_train,
                                         calculators=calc_sr,
                                         do_gradients=True,
                                         precompute = True,
                                         lazy_fill_up = False,
                                         batch_size=len(frames_train), 
                                         shuffle=False,
                                         collate_fn=_equistore_collate_fn)

single calculator passed


In [257]:
batch = next(iter(dataloader_train))

In [258]:
batch

[TensorMap with 1 blocks
 keys: _
       0,
 TensorMap with 1 blocks
 keys: _
       0]

In [211]:
from nn.aggregation import StructureWiseAggregation

In [212]:
model = BPNNModel(aggregation=StructureWiseAggregation(mode="sum"))
model.initialize_weights(batch[0])

In [224]:
dataloader_train = create_rascaline_dataloader(frames_train[:2],
                                         calculators=calc_sr,
                                         do_gradients=True,
                                         precompute = True,
                                         lazy_fill_up = False,
                                         batch_size=1, 
                                         shuffle=False), 

batch = next(iter(dataloader_train[0]))

single calculator passed


In [232]:
len(frames_train[0])

192

In [225]:
out = model(batch[0],batch[2])

grad_out = torch.autograd.grad(out.block(0).values, [block.values for key, block in batch[0].items()], grad_outputs=torch.ones_like(out.block(0).values))

In [228]:
grad_out[0].shape

torch.Size([384, 450])

In [160]:
grad_out[0].shape

torch.Size([384, 450])

In [184]:
grad_A = grad_out[0][:,:] # only consider first sample

In [172]:
#THIS IS BROKEN BECAUSE JOIN DOE NOT RESPECT STRUCTURE
grad_B = equistore.sum_over_samples_block(batch[0][0].gradient("positions"), sample_names=["sample"]).values

In [231]:
dataloader_train = create_rascaline_dataloader(frames_train[:2],
                                         calculators=calc_sr,
                                         do_gradients=True,
                                         precompute = True,
                                         lazy_fill_up = False,
                                         batch_size=1, 
                                         shuffle=False), 

batch = next(iter(dataloader_train[0]))

out = model(batch[0],batch[2])

grad_out = torch.autograd.grad(out.block(0).values, [block.values for key, block in batch[0].items()], grad_outputs=torch.ones_like(out.block(0).values))

grad_A = grad_out[0]

#THIS IS BROKEN BECAUSE JOIN DOE NOT RESPECT STRUCTURE
grad_B = equistore.sum_over_samples_block(batch[0][0].gradient("positions"), sample_names=["sample"]).values
grad_B = torch.cat([grad_B, grad_B], dim=0)


grad_A = grad_A.unsqueeze(1)  # Now t1 has shape (192, 1, 450)
grad_B = grad_B.transpose(1, 2)  # Now t2 has shape (192, 450, 3)
result = torch.bmm(grad_A, grad_B)  # result has shape (192, 1, 3)

f_out = result.squeeze(1)  # result now has shape (192, 3)





forces = -batch[2][1].positions

single calculator passed


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [190]:
f_out.shape

torch.Size([384, 3])

In [182]:
grad_B.shape

torch.Size([384, 3, 450])

In [166]:
batch[0][0].gradient("positions").samples

Labels(
    sample  structure  atom
      0         0       0
      0         0       6
      0         0       10
      0         0       13
      0         0       14
      0         0       17
      0         0       19
      0         0       21
      0         0       22
      0         0       31
      0         0       32
      0         0       35
      0         0       42
      0         0       45
      0         0       50
      0         0       60
      0         0       61
      0         0       64
      0         0       70
      0         0       74
      0         0       77
      0         0       78
      0         0       81
      0         0       83
      0         0       86
      0         0       95
      0         0       96
      0         0       99
      0         0      106
      0         0      114
      0         0      124
      0         0      127
      0         0      128
      0         0      131
      0         0      134
      0         0    

In [ ]:
grad_out[1].shape

In [ ]:
out = model(batch[0],batch[2])

In [ ]:
grad_out[0].shape

In [ ]:
equistore.sum_over_samples_block(batch[0][0].gradient("positions"), sample_names=["sample"]).values.shape

In [ ]:
grad_out[0].shape

In [ ]:


grad_out[0] * equistore.sum_over_samples_block(batch[0][0].gradient("positions"), sample_names=["sample"]).values.swapaxes(1,2)

In [ ]:
#Will be a tensormap aswell:
#dNNdF = 

# dNNdx = dNNdF * dFdx

# E = NN(F(x))



In [ ]:
grad_out[0].shape

In [ ]:
next((dataloader_train))

In [ ]:
batch

In [ ]:
feats, energies, forces, idx, systems = batch

In [ ]:
idx

In [ ]:
feats

In [ ]:
torch.unique(idx)

In [ ]:
class ModelNoEqui(torch.nn.Model):
    def __init__(self, n_in, n_out):
        super().__init__()
        
        self.model = torch.nn.Sequential(
            torch.nn.Linear(n_in,n_out)
        )
    
    def forward(self, feat, idx):
        atomic_energies = self.model(feat)
        structure_energies = torch.zeros_like(torch.unique(idx))
        structure_energies.index_add_(0, idx, atomic_energies)

        return structure_energies

In [ ]:
batch[0].block(species_center=1)

In [ ]:
feats, energies, forces, idx, systems = batch

In [ ]:
feats.shape

In [ ]:
energies

In [ ]:
forces.shape

In [ ]:
def inner_join() -> equistore.TensorMap:
    
    """
    Inner join of two TensorMaps.


    TensorMap_int = 





    """


def join_block() -> equistore.TensorBlock:
    
    """
    joins tensorblock along a given dimension

    handles logic of checking that samples or properties match ... etc
    """



def outer_join(maps: List[equistore.TensorMap], axis: str) -> equistore.TensorMap:
    
    """
    Outer join of TensorMaps.

    joins all blocks from TensorMaps with identical keys, along given dimension

    new blocks are returned in a new TensorMap.
    blocks with unique keys are kept as they are and are part of the new TensorMap

    
    ----------------------------------------------------

    Simple case, two tensor maps:
    outer_join(TensorMap_A, TensorMap_B) -> TensorMap_out


    ----------------------

    TensorMap_A:

    TensorMap with 2 blocks
    keys: species_center
                1
                8
    
    
    TensorMap_A.block(species_center=1):
    TensorBlock
        samples (100)

    TensorMap_A.block(species_center=8):
    TensorBlock
        samples (100)
    
     
    TensorMap_B:
    TensorMap with 2 blocks
    keys: species_center
                6
                8

    TensorMap_A.block(species_center=6):
    TensorBlock
        samples (100)

    TensorMap_A.block(species_center=8):
    TensorBlock
        samples (100)

    returns:
    
    TensorMap_out:
    TensorMap with 3 blocks
    keys: species_center
                1
                6
                8

    TensorMap_out.block(species_center=1):
    TensorBlock
        samples (100)
    
    TensorMap_out.block(species_center=6):
    TensorBlock
        samples (100)

    TensorMap_out.block(species_center=8):
    TensorBlock
        samples (200)

    ----------------------------------------------------

    

    int_tensorblock_holder = {}

    for map in tensormaps:
        for key, block in map.items():
            if key in int_tensorblock_holder:
                int_tensorblock_holder[key].append(block)
            else:
                int_tensorblock_holder[key] = [block]
    
    new_keys = []
    new_blocks = []

    for key, blocks in int_tensorblock_holder.items():
        new_blocks.append(join_block(blocks))
    
    
    TensorMap_out = TensorMap(new_keys, new_blocks)


    """


def inner_join(maps: List[equistore.TensorMap], axis: str) -> equistore.TensorMap:
    
    """
    Inner join of TensorMaps.

    Returns the joined Tensorblocks of the intersecting keys of all the TensorMaps

    new blocks are returned in a new TensorMap.

    
    ----------------------------------------------------

    Simple case, two tensor maps:
    inner_join(TensorMap_A, TensorMap_B) -> TensorMap_out


    ----------------------

    TensorMap_A:

    TensorMap with 2 blocks
    keys: species_center
                1
                8
    
    
    TensorMap_A.block(species_center=1):
    TensorBlock
        samples (100)

    TensorMap_A.block(species_center=8):
    TensorBlock
        samples (100)
    
     
    TensorMap_B:
    TensorMap with 2 blocks
    keys: species_center
                6
                8

    TensorMap_A.block(species_center=6):
    TensorBlock
        samples (100)

    TensorMap_A.block(species_center=8):
    TensorBlock
        samples (100)

    returns:
    
    TensorMap_out:
    TensorMap with 1 block
    keys: species_center
                8

    TensorMap_out.block(species_center=8):
    TensorBlock
        samples (200)

    ----------------------------------------------------

    

    int_tensorblock_holder = {}

    for map in tensormaps:
        for key, block in map.items():
            if key in int_tensorblock_holder:
                int_tensorblock_holder[key].append(block)
            else:
                int_tensorblock_holder[key] = [block]
    
    new_keys = []
    new_blocks = []

    for key, blocks in int_tensorblock_holder.items():
        if len(blocks) == len(tensormaps):
            new_blocks.append(join_block(blocks))
            new_keys.append(key)
    
    
    if len(new_keys) == 0:
        raise ValueError("No common keys found in TensorMaps") ?
    TensorMap_out = TensorMap(new_keys, new_blocks)


    """




In [ ]:
idx

In [ ]:
feat

In [ ]:
import numpy as np

In [ ]:
prop.block(0).samples

In [ ]:
prop.block(0).gradient("positions").samples

In [ ]:
feat.block(0).samples

In [ ]:
np.array(feat.block(0).samples.view("structure")).flatten()

In [ ]:
frames_water = load_PBE0_TS()

In [ ]:
energies = [frame.get_potential_energy() for frame in frames_water]

In [ ]:
a = torch.arange(60).reshape(20,3,1)
a

In [ ]:
a.flatten()

In [ ]:
torch.mean(torch.tensor(energies))

In [ ]:
torch.var(torch.tensor(energies))

In [ ]:
feat, prop, syst = next(iter(dataloader_train))

# define the trainer
module = BPNNRascalineModule(feat, transformer)#transformer)

# --- train the model ---
n = 10
logger = CSVLogger("logs", name="my_exp_name")


trainer = Trainer(max_epochs=n,
                  precision=64,
                  accelerator="cpu",
                  inference_mode=False,
                  num_sanity_val_steps=0,
                  logger=logger,
                  log_every_n_steps=50)

# create an empty text file and write losses to it later

In [ ]:
for i in range(5):
    feat, prop, syst = next(iter(dataloader_train))
    print(prop.block(0).values.mean())
    
    prop = transformer.forward(syst, prop)
    print(prop.block(0).values.mean())

    out = module(feat, syst)

    prop = transformer.inverse_transform(syst, prop)
    print(prop.block(0).values.mean())

In [ ]:
for i in range(1):

    with torch.enable_grad():
        out = module.forward(feat, syst)
        out = out.copy()

    with torch.no_grad():
        out = transformer.inverse_transform(syst, out)
        energy_mse, force_mse = module.loss_fn.report(out, prop)

In [ ]:
with torch.enable_grad():
    feat, prop, syst = next(iter(dataloader_val))
    out = module.forward(feat, syst)

    # this gives us (prediction on offset removed)

    print(out.block(0).values)

    out = transformer.inverse_transform(syst, out)
    energy_mse, force_mse = module.loss_fn.report(out, prop)

In [ ]:
out.block(0).gradient("positions").values